In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.width',1000)
from pydotplus import graphviz

In [5]:
# try:
#     abcedefg==1
#     print("Graphviz 已经添加坏境变量")
# except:
#     print("添加坏境变量 Graphviz ")
#     abcedefg=1
#     import os
#     os.environ["PATH"] += os.pathsep + r'C:/Program Files (x86)/Graphviz2.38/bin/'

In [6]:
dataset = pd.read_csv(r'data\watermelon_3.csv', index_col = 0)
TrueLabel = '是'
dataset

,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
编号,,,,,,,,,
1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是
6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,是
7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否


In [7]:
class Node(object):
    def __init__(self, attr_init = None, label_init = None, attr_down_init=None):
        '''
            definition of decision node class
            
        @param attr_init:     attribution as parent for a new branching
        @param attr_down:     dict {key, val}
                        key:    categoric: categoric attr_value
                                continuous: <= div_val for small part
                                            >  div_val for big part
                        value:  children
        @param label    :     class label (the majority of current sample labels)
        
        '''
        self.attr = attr_init
        self.label = label_init
        if attr_down_init is None:
            self.attr_down = dict()
        else:
            self.attr_down = attr_down_init
    def __str__(self):
        if self.attr is None:
            ret = 'Label = %s'%(str(self.label))
        else:
            ret = 'Label = %s, Attr = %s'%(str(self.label), self.attr)
        
        ret += str(self.attr_down)
        return ret

In [8]:
def Counter(array):
    '''
        calculating the appeared element and it's counts
        @param the list of element 
        @return: count : dict, the appeared element and it's counts
    
    '''
    count = dict()
    for e in array:
        count[e] = count.get(e, 0) + 1
    return count


In [9]:
def getLabelEnt(df):
    labelCount = Counter(df[df.columns[-1]])
    nSample = len(df)
    ent = 0
    for val in labelCount.values():
        p = val / nSample
        e = -p * np.log2(p) if p else 0
        ent += e
    return ent

In [10]:

def __InfoGain(entLabel, samedAttr_list, TrueLabel = TrueLabel):
    
    #print(samedAttr_list)
    numAttr_list = [len(attr) for attr in samedAttr_list]
    nSample = sum(numAttr_list)
    ent = 0
    for attr, num in zip(samedAttr_list, numAttr_list):
        trueCount = sum(attr[attr.columns[-1]] == TrueLabel)
        p = trueCount / num if trueCount else 0
        not_p = 1 - p
        ent -= num / nSample * (p*np.log2(p) + not_p*np.log2(not_p)) if p*not_p else 0
    return entLabel - ent


In [13]:
#info_gain = InfoEnt(df.values[:,-1])  # info_gain for the whole label

def InfoGain(df, index):
    '''
    calculating the information gain of an attribution
    
    @param df:        dataframe, the pandas dataframe of the dataset
    @param index:     the target attribution index in df
    @return info_gain:the information gain of the current classify
    @return div_val  :only valid in continuous attribution , return a real num for binary-division threshold
                      for discrete attribution , return 0
    '''
    entLabel = getLabelEnt(df)
    nSample = len(df)
    if df[index].dtype not in (float, int):

        AttrCount = Counter(df[index])
        attrs = list(AttrCount.keys())
        #print(AttrCount)
        samedAttr_list = [ df[df[index] == attr] for attr in attrs]
        return __InfoGain(entLabel, samedAttr_list), 0, attrs, samedAttr_list
    else:
        attr_val = sorted(list(df[index]))
        Ta = [(i+j)/2 for i,j in zip(attr_val[:-1], attr_val[1:])]
        best_threshold = 0
        best_gain = 0
        best_samedAttr_list = None
        for threshold in Ta:
            small = df[df[index] <= threshold]
            big = df[df[index] > threshold]
            AttrCount = [len(small), len(big)]
            samedAttr_list = [small, big]
            gain_t = __InfoGain(entLabel, samedAttr_list)
            if gain_t > best_gain:
                best_gain = gain_t
                best_threshold = threshold
                best_samedAttr_list = samedAttr_list
        attrs = ['<= %.3f'%(best_threshold), '> %.3f'%(best_threshold)]
        return best_gain, best_threshold, attrs, best_samedAttr_list
            
for attr in dataset.columns[:-1]:
    gain, div_val, _,_ = InfoGain(dataset,attr)
    print("The Gain of %3s is %.3f and best threshold is %.3f"%(attr, gain, div_val))

The Gain of  色泽 is 0.108 and best threshold is 0.000
The Gain of  根蒂 is 0.143 and best threshold is 0.000
The Gain of  敲声 is 0.141 and best threshold is 0.000
The Gain of  纹理 is 0.381 and best threshold is 0.000
The Gain of  脐部 is 0.289 and best threshold is 0.000
The Gain of  触感 is 0.006 and best threshold is 0.000
The Gain of  密度 is 0.262 and best threshold is 0.381
The Gain of 含糖率 is 0.349 and best threshold is 0.126


In [23]:
def OptAttr(df, usedAttr = None):
    info_gain, div_val =0, 0
    opt_attr = None
    opt_attr_val = None
    opt_blocks = None
    for attr in df.columns[:-1]:
        if usedAttr and attr in usedAttr: continue
        gain_tmp, div_tmp, attr_val, blocks = InfoGain(df,attr)
        if gain_tmp > info_gain:
            info_gain = gain_tmp
            div_val = div_tmp
            opt_attr = attr
            opt_attr_val = attr_val
            opt_blocks = blocks
    return opt_attr, div_val, opt_attr_val, opt_blocks

opt = OptAttr(dataset)
opt

('纹理', 0, ['清晰', '稍糊', '模糊'], [    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率 好瓜
  编号                                         
  1   青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460  是
  2   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376  是
  3   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264  是
  4   青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318  是
  5   浅白  蜷缩  浊响  清晰  凹陷  硬滑  0.556  0.215  是
  6   青绿  稍蜷  浊响  清晰  稍凹  软粘  0.403  0.237  是
  8   乌黑  稍蜷  浊响  清晰  稍凹  硬滑  0.437  0.211  是
  10  青绿  硬挺  清脆  清晰  平坦  软粘  0.243  0.267  否
  15  乌黑  稍蜷  浊响  清晰  稍凹  软粘  0.360  0.370  否,
      色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率 好瓜
  编号                                         
  7   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149  是
  9   乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  0.666  0.091  否
  13  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161  否
  14  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198  否
  17  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103  否,
      色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率 好瓜
  编号                                         
  11  浅白  硬挺  清脆  模糊  平坦  硬滑  0.245  0.057  否
  1

In [27]:
#[df[opt[0]].head[0] for df in opt[2]]


In [28]:
d = {'a':4,'b':2,'c':3}
sorted(d.items(), key = lambda x : x[1], reverse=True)[0][0]

'a'

In [29]:
def TreeGenerate(df, usedAttr=set()):
    '''
    Branching for decision tree using recursion
    @param  df  : the pandas dataframe of the dataset
    @return root: Node, the root node of decision tree
    '''
    #generating a ew root node
    new_node = Node()

    label_arr = df[df.columns[-1]]
    
    label_count = Counter(label_arr)
    node_label = sorted(label_count.items(), key = lambda x:x[1], reverse=True)[0][0]
    if label_count:
        # end if there is onlly 1 class in current node data
        # end if attriubtion array is empty
        new_node.label = node_label
        if len(label_count) <= 1:
            
            return new_node
        
        #get the optimal attriubtion for a new branching
        new_node.attr, div_val, attrs, blocks = OptAttr(df, usedAttr)
        
        if new_node.attr is None:
            return new_node
        
        
        print("The attribution of node is %s and best threshold is %.3f"%(new_node.attr, div_val))
        #print(div_res)

        print(attrs)
        for i in range(len(attrs)):
            attr = attrs[i]
            data = blocks[i]
            usedAttr.add(new_node.attr)
            print('used',usedAttr)
            new_node.attr_down[attr] = TreeGenerate(data)
            usedAttr.discard(new_node.attr)
        #print(new_node.attr_down.keys())
    return new_node

myDT = TreeGenerate(dataset)

The attribution of node is 纹理 and best threshold is 0.000
['清晰', '稍糊', '模糊']
used {'纹理'}
The attribution of node is 密度 and best threshold is 0.381
['<= 0.381', '> 0.381']
used {'密度', '纹理'}
used {'密度', '纹理'}
used {'纹理'}
The attribution of node is 触感 and best threshold is 0.000
['软粘', '硬滑']
used {'触感', '纹理'}
used {'触感', '纹理'}
used {'纹理'}


In [13]:
#myDT.attr_down['fuzzy'].label

In [14]:
# def preorder(root):
#     if root.attr is None:
#         print(root.label)
#     else:
#         print(root.attr)
#     for attr, subtree in root.attr_down.items():
#         print("->",attr)
#         preorder(subtree)

# preorder(myDT)

In [15]:
# import graphviz
# def TreeToGraph(i, g, root):
#     '''
#     build a graph from root on
#     @param i : node number in this tree
#     @prarm g : pydotplus.graphviz.Dot() object
#     @prarm root: the root node
    
#     @return i: node number after modified
#     @return g: pydotplus.graphviz.Dot() object after modified
#     @return g_node: the current root node in graphviz
#     '''
#     if root.attr == None:
#         g_node_label= u'Node:%d  Label:%s'%(i,root.label)
#     else:
#         g_node_label= u"Node:%d  Label : %s, Attribute : %s" % (i, root.label, root.attr)
#     #g_node_label = bytes(g_node_label,'ascii')
#     print(i,' ',g_node_label)
#     g_node = i
#     g.node(str(g_node), g_node_label)
    
#     for attr, subtree in root.attr_down.items():
#         i, g_child = TreeToGraph(i+1, g, subtree)
#         g.edge(str(g_node), str(g_child), label = attr)
#     return i, g_node

# def DrawPNG(root, out_file='myDT'):
#     '''
#     visualization of decision tree from root.
#     @param root: Node, the root node for tree.
#     @param out_file: str, name and path of output file
#     '''
        
#     g = graphviz.Digraph()  # generation of new dot   

#     TreeToGraph(0, g, root)
    
    
#     g.render(out_file)
#     g.view()
#     print("your The graph of decision tree has been saved to ",out_file)
#     return g

# pic_data = DrawPNG(myDT)


In [30]:
import re

In [31]:
def Predict(root, df_sample):
    while root.attr != None:
        sample_val = df_sample[root.attr].values[0]
        print("DT attr = %s, sample val = %s"%(root.attr, str(sample_val)))
        if type(sample_val) == str:
            if sample_val in root.attr_down:
                root = root.attr_down[sample_val]
            else:
                print("The key of sample ",sample_val,"%s isn't belone to attribute %s"%(str(type(sample_val)), root.attr))
                break
        else:
            key = list(root.attr_down.keys())
            try:
                div_val = float(re.findall(r"\d+.?\d*",key[0])[0])
            except:
                print("the type of sample is unmatch to the DT attr")
                break
            if sample_val <= div_val:
                root = root.attr_down[key[0]]
            else:
                root = root.attr_down[key[1]]
    return root.label
test = dataset[:1]
print(test)
Predict(myDT, test)

    色泽  根蒂  敲声  纹理  脐部  触感     密度   含糖率 好瓜
编号                                        
1   青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.46  是
DT attr = 纹理, sample val = 清晰
DT attr = 密度, sample val = 0.6970000000000001


'是'

In [32]:
#list(myDT.attr_down['distinct'].attr_down.keys())

In [33]:
def DrawPNG(root, out_file):
    '''
    visualization of decision tree from root.
    @param root: Node, the root node for tree.
    @param out_file: str, name and path of output file
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")
        
    g = graphviz.Dot()  # generation of new dot   

    TreeToGraph(0, g, root)
    g2 = graphviz.graph_from_dot_data( g.to_string() )
    
    g2.write_png(out_file) 

    
def TreeToGraph(i, g, root):
    '''
    build a graph from root on
    @param i: node number in this tree
    @param g: pydotplus.graphviz.Dot() object
    @param root: the root node
    
    @return i: node number after modified  
#     @return g: pydotplus.graphviz.Dot() object after modified
    @return g_node: the current root node in graphviz
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")

    if root.attr == None:
        g_node_label = "Node:%d\n好瓜:%s" % (i, root.label)
    else:
        g_node_label = "Node:%d\n好瓜:%s\n属性:%s" % (i, root.label, root.attr)
    g_node = i
    g.add_node( graphviz.Node( g_node, label = g_node_label, fontname="Microsoft Yahei" ) )
    
    for value in list(root.attr_down):
        i, g_child = TreeToGraph(i+1, g, root.attr_down[value])
        g.add_edge( graphviz.Edge(g_node, g_child, label = value, fontname="Microsoft Yahei") ) 

    return i, g_node

DrawPNG(myDT, 'myDT.png')